In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 출력을 위한 글꼴 설정
plt.rcParams['axes.unicode_minus'] = False    # 문자 - (마이너스) 정상 출력을 위한 코드

# 컨텐츠기반 추천시스템을 위한 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#실수 설정 옵션 (소수 셋째 짜리까지)
pd.options.display.float_format = '{:.3f}'.format

## 데이터셋

In [2]:
df_closet=pd.read_excel('C:/Users/82102/jupi python/프리온보딩/기술과제 제출/클로젯셰어_과제DB 대체 자료.xlsx')

In [3]:
df_closet.head()

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,NaN
2,Danha,embroidery point oriental mini skirt blue,a1140859,2021-01-02,2021-01-15,4days,B,NaN
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,NaN
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y


## 결측치 

In [4]:
#app_yn n 채우자
df_closet["app_yn"].fillna('N', inplace = True)
df_closet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1442 non-null   object        
 1   goodsnm     1471 non-null   object        
 2   USER        1471 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1471 non-null   datetime64[ns]
 5   p_type      1471 non-null   object        
 6   sub_type    1471 non-null   object        
 7   app_yn      1471 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 92.1+ KB


In [5]:
# brandnm, reg_date 결측치 확인
brandnm_nl=df_closet.loc[df_closet['brandnm'].isnull()]
reg_date_nl = df_closet.loc[df_closet['reg_date'].isnull()]

In [6]:
# reg_date 결측치
# 유저, 브랜드, 다른 피쳐가 거의 다르다 
reg_date_nl

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
265,Gucci,mamont chain bag red,a1144008,NaT,2021-01-21,7days,BA,Y
351,Lanvin,banding point jumper skyblue,a1142855,NaT,2021-01-18,7days,O,N
564,Gucci,leather point cross bag,a1146490,NaT,2021-02-18,4days,BA,Y
746,Fendi,monogram zucchino shoulder bag,a1145478,NaT,2021-02-04,4days,BA,N
1254,Fendi,back to school bag 1,a1149094,NaT,2021-03-12,7days,BA,Y


In [7]:
#결측값 정리 파일
df_tp=pd.read_csv('TP.csv')

In [8]:
df_tp= df_tp.rename(columns={
    '브랜드 결측값 브랜드 후보':'brandnm'
})

In [9]:
df_tp_dict=df_tp.to_dict()

In [10]:
for i in  list(df_tp_dict.values()):
    df_closet['brandnm'] = df_closet['brandnm'].fillna(i)

In [11]:
df_closet.replace(' ', np.nan, inplace = True)

In [12]:
df_closet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471 entries, 0 to 1470
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   brandnm     1469 non-null   object        
 1   goodsnm     1471 non-null   object        
 2   USER        1471 non-null   object        
 3   reg_date    1466 non-null   datetime64[ns]
 4   order_date  1471 non-null   datetime64[ns]
 5   p_type      1471 non-null   object        
 6   sub_type    1471 non-null   object        
 7   app_yn      1471 non-null   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 92.1+ KB


## 주문 sub_type 별로 컷

In [13]:
df_closet_ba = df_closet[df_closet['sub_type']=='BA']
df_closet_o = df_closet[df_closet['sub_type']=='O']
df_closet_d = df_closet[df_closet['sub_type']=='D']
df_closet_t = df_closet[df_closet['sub_type']=='T']
df_closet_b = df_closet[df_closet['sub_type']=='B']
df_closet_tp = df_closet[df_closet['sub_type']=='TP']

### bottom 확인

In [15]:
df_closet_b_list

['embroidery point oriental mini skirt blue',
 'pocket basic pattern slacks black',
 'slit basic pattern skirt white',
 'check pattern unbalance skirt ivory',
 'basic pattern skirt grey',
 'heart pattern quilted skirts',
 'pocket basic pattern wide-pants navy',
 'roll up basic pattern slacks navy',
 'basic pattern unbalance skirt skyblue',
 'pocket graphics pattern sweat-pants white',
 'pocket basic pattern wool slacks grey',
 'pocket basic pattern slacks navy',
 'pocket basic pattern skirt blue',
 'button check pattern pants',
 'pleats pocket wool skirt brown',
 'pleats basic pattern pleats-skirt grey',
 'basic pattern skirt grey',
 'quilting detail denim skirt',
 'pleats basic pattern wool pleats-skirt brown',
 'pattern pointed a-line skirts',
 'pocket basic pattern skirt white',
 'button basic pattern pants brown',
 'slit basic pattern skirt black',
 'paisley print pattern skirt',
 'pocket check pattern slacks charcoal',
 'pocket stripe pattern slacks black',
 'button check pattern 

In [18]:
#goodsnm 벡터화
tfidf = TfidfVectorizer()

tfidf_matrix_bottom = tfidf.fit_transform(df_closet_b['goodsnm'])
print(tfidf_matrix_bottom.shape)

(58, 69)


In [19]:
print(tfidf_matrix_bottom)

  (0, 5)	0.39048674831878877
  (0, 54)	0.15804822793757034
  (0, 33)	0.4638129375111069
  (0, 36)	0.4638129375111069
  (0, 44)	0.4638129375111069
  (0, 16)	0.4209198665128158
  (1, 4)	0.5759963158662922
  (1, 57)	0.5120829294138277
  (1, 39)	0.25893853616766477
  (1, 2)	0.33817984354418973
  (1, 43)	0.4739035191040322
  (2, 65)	0.6129966546870037
  (2, 58)	0.6470106706227533
  (2, 39)	0.21743905318077447
  (2, 2)	0.28398053867678286
  (2, 54)	0.27872496344520375
  (3, 25)	0.5848835771774631
  (3, 63)	0.5848835771774631
  (3, 10)	0.47505325893941164
  (3, 39)	0.18467769224081143
  (3, 54)	0.23672970547829816
  (4, 23)	0.7899825583617416
  (4, 39)	0.29401202353489014
  (4, 2)	0.3839866463706227
  (4, 54)	0.3768802766266753
  :	:
  (53, 26)	0.6841117393707405
  (53, 39)	0.18185929214251664
  (53, 2)	0.23751253047939874
  (53, 43)	0.3328348101585862
  (53, 5)	0.5759575617426456
  (54, 35)	0.6481398792132462
  (54, 57)	0.47704565243440006
  (54, 39)	0.24122167686376425
  (54, 2)	0.315041206

In [20]:
#코사인 유사도 측정
cosine_matrix_bottom = cosine_similarity(tfidf_matrix_bottom, tfidf_matrix_bottom)

In [21]:
cosine_matrix_bottom.shape

(58, 58)

### subtype 별 cosine 유사도 함수화

In [92]:
#
def Cosine_Subtype(df_x, idx): #df_x = sub_type별 데이터프레임, idx = goodsnm 품목 명
    tfidf_matrix_sub = tfidf.fit_transform(df_x['goodsnm'])
    cosine_matrix_sub = cosine_similarity(tfidf_matrix_sub,tfidf_matrix_sub)
    np.round(cosine_matrix_sub,4)
    #딕셔너리 생성
    goods_id = {}
    for i, c in enumerate(df_x['goodsnm']): goods_id[i] = c
    #아이디 별 굿즈 이름
    id2goods = {}
    for o, p in goods_id.items(): id2goods[p] =o
    
    idx_ = id2goods[idx]
    sim_scores = [(y,z) for y, z in enumerate(cosine_matrix_sub[idx_]) if y !=idx_] 
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = [(goods_id[w], score) for w , score in sim_scores[0:15]]
    
   #중복 제거 
    my_sim_scores = []
    for v in sim_scores:
        if v not in my_sim_scores:
            my_sim_scores.append(v)
    return my_sim_scores[0:5]

## bottom dataframe skirt, 나머지로 세분화
- skirt와 바지를 구분할 필요가 있다
- metric 점수를 기준으로 세워도 치마가 높은 순위로 나타날 수 있음
- 데이터 프레임을 쪼개는게 나을듯! (skrit, 나머지)

In [98]:
df_closet_skirt = df_closet_b.loc[df_closet_b['goodsnm'].str.contains('skirt' or 'skirts')]

In [100]:
df_closet_pants = df_closet_b.loc[df_closet_b['goodsnm'].str.contains('skirt' or 'skirts')==False]

In [101]:
df_closet_pants

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
65,Love Moschino,pocket basic pattern slacks black,a1141949,2021-01-10,2021-01-10,4days,B,Y
208,Sandro,pocket basic pattern wide-pants navy,a1143880,2021-01-20,2021-01-26,4days,B,Y
215,Thom Browne,roll up basic pattern slacks navy,a1144523,2021-01-26,2021-01-26,7days,B,N
258,Versace,pocket graphics pattern sweat-pants white,a1143701,2021-01-18,2021-01-22,4days,B,Y
364,Calvin Klein,pocket basic pattern wool slacks grey,a1142677,2021-01-13,2021-01-18,4days,B,Y
381,Audrey Tailor,pocket basic pattern slacks navy,a1147589,2021-02-25,2021-02-28,7days,B,Y
498,Burberry London,button check pattern pants,a1143498,2021-01-17,2021-02-22,4days,B,Y
864,Banana Republic,button basic pattern pants brown,a1151213,2021-03-29,2021-03-31,4days,B,Y
909,At Corner,pocket check pattern slacks charcoal,a1146721,2021-02-17,2021-03-29,4days,B,Y
989,Balenciaga,pocket stripe pattern slacks black,a1132521,2020-11-23,2021-03-24,4days,B,Y


In [93]:
#예시
Cosine_Subtype(df_closet_b,'pocket basic pattern slacks black' )

[('pocket basic pattern slacks black', 1.0000000000000002),
 ('pocket stripe pattern slacks black', 0.6538873230543492),
 ('pocket basic pattern slacks navy', 0.6225073330421497),
 ('ribbon basic pattern slacks black', 0.6214580547261949),
 ('buckle basic pattern slacks black', 0.5905248692538959)]

### 가방

In [94]:
df_closet_ba

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
0,Gucci,flower dionysus shoulder bag beige,a161237,2019-12-06,2021-01-15,7days,BA,Y
8,Gucci,dionysus cross bag,a1143028,2021-01-14,2021-01-15,4days,BA,Y
10,Louis Vuitton,twist monogram patent shoulder bag black,a1140216,2020-12-28,2021-01-15,7days,BA,Y
11,Tory Burch,flower ethnic pattern cross bag white,a1141255,2021-01-05,2021-01-15,4days,BA,Y
14,Chloe,faye small bag black,a1129386,2020-11-09,2021-01-15,4days,BA,Y
...,...,...,...,...,...,...,...,...
1458,Chanel,boy chanel clutch black,a1147960,2021-03-01,2021-03-01,7days,BA,Y
1459,Gucci,marmont gg logo quilted bag Black 2,a177034,2020-01-23,2021-03-01,4days,BA,Y
1463,Burberry,check mackenzie cross bag pink,a1147938,2021-03-01,2021-03-01,7days,BA,Y
1466,Yves Saint Laurent,logo point clutch bag pink,a119721,2019-04-10,2021-03-01,7days,BA,Y


In [95]:
Cosine_Subtype(df_closet_ba, 'dionysus cross bag')

[('dionysus cross bag', 1.0),
 ('dionysus cross bag black', 0.9378765143550344),
 ('dionysus bag', 0.8678766041315753),
 ('dionysus cross bag beige', 0.7982556599047357)]

In [96]:
Cosine_Subtype(df_closet_ba, 'boy chanel clutch black')

[('boy chanel clutch black', 1.0),
 ('boy chanel crossbody bag black', 0.7214143642829653),
 ('quilted boy chain bag black', 0.4744386469117857),
 ('Y clutch black', 0.4675751443032589)]

In [97]:
Cosine_Subtype(df_closet_ba, 'marmont gg logo quilted bag Black 2')

[('marmont gg logo quilted bag Black 4', 1.0000000000000002),
 ('marmont gg logo quilted bag Black 2', 1.0000000000000002),
 ('marmont GG logo quilted bag black', 1.0000000000000002),
 ('marmont gg logo quilted bag Black', 1.0000000000000002),
 ('quilted GG marmont clutch bag', 0.7751199815864652)]

In [102]:
Cosine_Subtype(df_closet_skirt,'slit basic pattern skirt white' )

[('slit basic pattern skirt white', 1.0),
 ('slit basic pattern skirt black', 0.6311085295193565),
 ('pocket basic pattern skirt white', 0.6168252033103185),
 ('front slit patterned skirt', 0.32663020205986104),
 ('basic pattern skirt grey', 0.30891624803710194)]

- 가방의 경우 같은 종류가 많다 보니 점수가 후하게 잘 나오는 편
- 상위 5개로 끊었을 때 비슷한 종류 4개, 1개의 다른 종류 (색상, 다른 것 동일)

### 아우터 

In [103]:
df_closet_o

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
1,El estilo de Claire,beads cropped jacket black,a1140859,2021-01-02,2021-01-15,4days,O,N
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,2021-01-01,2021-01-15,7days,O,Y
5,Prada,ruffle detail long coat,a110231,2018-10-18,2021-01-15,7days,O,Y
6,Max Mara,strap basic pattern wrap-coat black,a1139960,2020-12-26,2021-01-15,7days,O,Y
7,Molliolli,pocket basic pattern fur-outer ivory,a115159,2019-01-29,2021-01-15,4days,O,Y
...,...,...,...,...,...,...,...,...
1460,Chanel,brooch check pattern jacket cream,a1146108,2021-02-11,2021-03-01,7days,O,Y
1461,Chanel,pocket tweed pattern cashmere coat pink,a1147959,2021-03-01,2021-03-01,7days,O,N
1462,Max Mara,zipper detail two tone jacket,a112319,2018-11-29,2021-03-01,4days,O,Y
1465,London Fog,button basic pattern coat grey,a1147932,2021-03-01,2021-03-01,4days,O,Y


In [111]:
Cosine_Subtype(df_closet_o, 'pocket tweed pattern cashmere coat pink' )

[('pocket tweed pattern cashmere coat pink', 1.0),
 ('pocket basic pattern jacket pink', 0.510114219434768),
 ('strap basic pattern cashmere coat black', 0.45584975051057364),
 ('button basic pattern cashmere singlebreasted-coat black',
  0.42839300140390035),
 ('belt basic pattern cashmere coat grey', 0.42742893363077755)]

- 아우터의 같은 경우 metric 기준선을 생각해놓는 것이 좋을 듯하다
- 다만 그렇게 되면 추천 항목수가 현저히 줄어들을 것 같다 이부분은 얘기를 해보자

### top

In [108]:
df_closet_t

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
3,Danha,graphics pattern top pink,a1140859,2021-01-02,2021-01-15,4days,T,N
22,& Other Stories,basic pattern sweater ivory,a1143019,2021-01-14,2021-01-14,4days,T,N
24,Balmain,basic pattern sweat-shirts white,a1142679,2021-01-13,2021-01-14,7days,T,Y
28,Chanel,brooch stripe pattern turtle neck turtleneck navy,a1136028,2020-12-09,2021-01-14,4days,T,Y
49,Yves Saint Laurent,basic pattern wool sweater black,a1129563,2020-11-09,2021-01-12,4days,T,Y
...,...,...,...,...,...,...,...,...
1443,Gucci,printing lettering pattern t-shirts cream,a1147907,2021-03-01,2021-03-02,4days,T,Y
1451,Thom Browne,button stripe pattern shirts white,a1117218,2020-08-30,2021-03-02,7days,T,Y
1464,Burberry London,embroidery basic pattern wool sweater brown,a1147932,2021-03-01,2021-03-01,4days,T,Y
1468,Burberry,basic pattern sweater beige,a1147848,2021-02-28,2021-03-01,7days,T,Y


In [109]:
Cosine_Subtype(df_closet_t, 'basic pattern sweater ivory' )

[('basic pattern sweater ivory', 1.0000000000000002),
 ('basic pattern t-shirts ivory', 0.7845870426672512),
 ('embroidery basic pattern sweater ivory', 0.7843532077976625),
 ('button basic pattern blouse ivory', 0.6597007271688048),
 ('basic pattern sweat-shirts ivory', 0.6492418092286076)]

- 남녀 구분 어떡하지..?

### 드레스

In [112]:
df_closet_d

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
19,El estilo de Claire,broach shirt layered dress brown,a1141367,2021-01-06,2021-01-14,4days,D,N
20,Mojosphine,Geometric pattern ribbon belt dress,a1141367,2021-01-06,2021-01-14,4days,D,N
29,Sandro,ruffle basic pattern A-Line dress black,a1136028,2020-12-09,2021-01-14,4days,D,Y
39,Plastic Island,button flower pattern A-Line dress ivory,a13934,2018-03-27,2021-01-13,4days,D,Y
40,Gucci,brooch check pattern H-Line dress ivory,a1141580,2021-01-07,2021-01-13,4days,D,Y
...,...,...,...,...,...,...,...,...
1439,Jill Stuart,romantic sleeve dress pink,a1147531,2021-02-25,2021-03-02,4days,D,Y
1440,& Other Stories,button flower pattern A-Line wrap-dress yellow,a1147531,2021-02-25,2021-03-02,4days,D,Y
1441,Zooc,stripe pattern A-Line shirt-dress yellow,a1147531,2021-02-25,2021-03-02,4days,D,Y
1444,On&On,gather basic pattern A-Line dress cream,a1115495,2020-08-22,2021-03-02,4days,D,Y


In [115]:
Cosine_Subtype(df_closet_d, 'ruffle basic pattern A-Line dress black')

[('ruffle basic pattern A-Line dress black', 1.0),
 ('basic pattern H-Line dress black', 0.726699236164849),
 ('basic pattern A-Line dress black', 0.726699236164849),
 ('ruffle basic pattern A-Line dress pink', 0.7261053310342067),
 ('ruffle lace dress black', 0.7198467579966914)]

In [114]:
Cosine_Subtype(df_closet_d, 'Geometric pattern ribbon belt dress')

[('geometric pattern H-Line dress black', 0.5377238661976531),
 ('geometric pattern H-Line dress brown', 0.47962837286059024),
 ('geometric pattern A-Line dress purple', 0.41591463047319915),
 ('black line point belt dress white', 0.3990815015392837),
 ('tweed belt set dress', 0.38028562096885066)]

- metric점수의 기준선에 대해 팀원과 회의하자
- 너무 낮은 경우만 존재하는 품목도 존재

## 총 구분 데이터 프레임
- BA
    - df_closet_ba 
- Outer
    - df_closet_o 
- Dress
    - df_closet_d 
- Top
    - df_closet_t 
- Bottom
    - df_closet_b 
        - df_closet_skirt
        - df_closet_pants
- Two
    - df_closet_tp